In [3]:
!pip install mlflow

     |████████████████████████████████| 10.4MB 2.7MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 92kB 9.7MB/s 
     |████████████████████████████████| 153kB 45.3MB/s 
     |████████████████████████████████| 6.0MB 44.4MB/s 
     |████████████████████████████████| 460kB 41.3MB/s 
     |████████████████████████████████| 1.1MB 42.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 204kB 43.1MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=884ad5c3a3cb45264434cd12e7c4997a9b141a7544f6497f2a46ba4df27061c5
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Succes

In [0]:
import pandas as pd
import numpy as np
from datetime  import datetime  
from datetime import timedelta  
import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.lightgbm
import seaborn as sns

In [0]:
np.random.seed(0)

In [0]:
data_folder = '/content/drive/My Drive/Kaggle_M5/00_data/'
submission = pd.read_csv(data_folder + 'sample_submission.csv')

In [0]:
params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective':  'tweedie',
        'n_jobs': -1,
        'seed': 0,
        'learning_rate': 0.05, 
        'bagging_fraction': 0.9,
        'bagging_freq': 1, 
        'colsample_bytree': 0.75,
        'num_iterations': 5000,
        'early_stopping_round': 500,
        'max_bin': 100,
        'boost_from_average': False}

In [0]:
params = {'bagging_fraction': 0.5, 
          'bagging_freq': 10, 
          'boosting_type': 'gbdt', 
          'colsample_bytree': 0.75, 
          'early_stopping_round': 500, 
          'learning_rate': 0.05, 
          'metric': 'rmse', 
          'n_jobs': -1, 
          'num_iterations': 5000, 
          'early_stopping_round': 500,         
          'objective': 'tweedie', 
          'seed': 0}

In [0]:
param_grid = {'boosting_type': ['gbdt', 'dart'],
        'metric': ['rmse'],
        'objective': ['poisson', 'tweedie'],
        'n_jobs': [-1],
        'seed': [0],
        'learning_rate':  [0.05, 0.075],
        'bagging_fraction': [0.75],
        'bagging_freq': [5, 10], 
        'colsample_bytree': [0.75],
        'num_iterations': [5000],
        'early_stopping_round': [500]}

In [0]:
evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation_rows = [row for row in submission['id'] if 'validation' in row] 
validation_csv = submission[submission['id'].isin(validation_rows)]


In [0]:
not_features = ['d', 'id', 'item_id', 'store_id', 'demand', 'date', 'start_date']

In [0]:
validation = pd.DataFrame(columns=submission.columns)
data_folder = '/content/drive/My Drive/Kaggle_M5/01_preprocessed_data/'

overall_rmse = 0
pred_true = pd.DataFrame(columns=['pred', 'true'])
rmse_score = []
mae_score = []
j = 0
for param in ParameterGrid(param_grid):
  print("----------------------")
  print(j)
  print(param)
  for i in ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']:
      print(i)
      
      X_val = pd.read_pickle(data_folder + "X_val_{}.pkl".format(i))
      X_test = pd.read_pickle(data_folder + "X_test_{}.pkl".format(i))
      X_train = pd.read_pickle(data_folder + "X_train_{}.pkl".format(i))
      y_train = X_train['demand']
      y_val = X_val['demand']
      print(X_train.shape)
      features = X_train.columns[~X_train.columns.isin(not_features) ]

      train_set = lgb.Dataset(X_train[features],y_train)
      val_set = lgb.Dataset(X_val[features], y_val)
      model = lgb.train(param, train_set, valid_sets = [train_set, val_set], verbose_eval = 100)
      
      val_pred = model.predict(X_val[features])
      val_score = metrics.mean_absolute_error(val_pred, y_val)

      temp = pd.DataFrame({'pred':val_pred, 'true':y_val})
      pred_true =pred_true.append(temp)
      print(f'Our val mae score is {val_score}')

      y_test = model.predict(X_test[features])
      X_test['demand'] = y_test
      predictions = X_test[['id', 'days_from_start', 'demand']]
      predictions = pd.pivot(predictions, index = 'id', columns = 'days_from_start', values = 'demand').reset_index()
      predictions.columns = ['id'] + ['F' + str(j + 1) for j in range(28)]
      validation = validation.append(predictions)
  rmse = np.sqrt(np.mean((pred_true['pred'] - pred_true['true'])**2) )
  rmse_score.append(rmse)
  mae_score.append(np.mean(np.abs(pred_true['pred'] - pred_true['true']) ))
  j = j + 1
    

----------------------
0
{'bagging_fraction': 0.75, 'bagging_freq': 5, 'boosting_type': 'gbdt', 'colsample_bytree': 0.75, 'early_stopping_round': 500, 'learning_rate': 0.05, 'metric': 'rmse', 'n_jobs': -1, 'num_iterations': 5000, 'objective': 'poisson', 'seed': 0}
CA_1
(4510808, 46)


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 2.78533	valid_1's rmse: 2.24759
[200]	training's rmse: 2.71834	valid_1's rmse: 2.20953
[300]	training's rmse: 2.67628	valid_1's rmse: 2.19993
[400]	training's rmse: 2.63692	valid_1's rmse: 2.18155
[500]	training's rmse: 2.60285	valid_1's rmse: 2.17362
[600]	training's rmse: 2.57496	valid_1's rmse: 2.16809
[700]	training's rmse: 2.54864	valid_1's rmse: 2.16504
[800]	training's rmse: 2.5327	valid_1's rmse: 2.16549
[900]	training's rmse: 2.51406	valid_1's rmse: 2.16536
[1000]	training's rmse: 2.49294	valid_1's rmse: 2.16242
[1100]	training's rmse: 2.47869	valid_1's rmse: 2.16177
[1200]	training's rmse: 2.46517	valid_1's rmse: 2.16115
[1300]	training's rmse: 2.45361	valid_1's rmse: 2.16115
[1400]	training's rmse: 2.44082	valid_1's rmse: 2.16693
[1500]	training's rmse: 2.43155	valid_1's rmse: 2.17632
[1600]	training's rmse: 2.42085	valid_1's rmse: 2.17862
[1700]	training's rmse: 2.41047	valid_1's rmse: 2.1

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 2.20559	valid_1's rmse: 2.14394
[200]	training's rmse: 2.16759	valid_1's rmse: 2.11163
[300]	training's rmse: 2.14553	valid_1's rmse: 2.10674
[400]	training's rmse: 2.12555	valid_1's rmse: 2.10376
[500]	training's rmse: 2.10969	valid_1's rmse: 2.09599
[600]	training's rmse: 2.09531	valid_1's rmse: 2.09106
[700]	training's rmse: 2.08366	valid_1's rmse: 2.08964
[800]	training's rmse: 2.07227	valid_1's rmse: 2.08543
[900]	training's rmse: 2.0603	valid_1's rmse: 2.08124
[1000]	training's rmse: 2.05148	valid_1's rmse: 2.07544
[1100]	training's rmse: 2.04272	valid_1's rmse: 2.07138
[1200]	training's rmse: 2.03596	valid_1's rmse: 2.06846
[1300]	training's rmse: 2.02883	valid_1's rmse: 2.068
[1400]	training's rmse: 2.02249	valid_1's rmse: 2.06199
[1500]	training's rmse: 2.0159	valid_1's rmse: 2.05609
[1600]	training's rmse: 2.01002	valid_1's rmse: 2.0544
[1700]	training's rmse: 2.00395	valid_1's rmse: 2.05336

...

Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 2.97724	valid_1's rmse: 3.12214
[200]	training's rmse: 2.91406	valid_1's rmse: 2.99074
[300]	training's rmse: 2.86551	valid_1's rmse: 2.95283
[400]	training's rmse: 2.82338	valid_1's rmse: 2.95668
[500]	training's rmse: 2.78538	valid_1's rmse: 2.94689
[600]	training's rmse: 2.75739	valid_1's rmse: 2.94161
[700]	training's rmse: 2.729	valid_1's rmse: 2.93635
[800]	training's rmse: 2.70583	valid_1's rmse: 2.93464
[900]	training's rmse: 2.6823	valid_1's rmse: 2.92635
[1000]	training's rmse: 2.66202	valid_1's rmse: 2.93081
[1100]	training's rmse: 2.64431	valid_1's rmse: 2.93317
[1200]	training's rmse: 2.62457	valid_1's rmse: 2.932
[1300]	training's rmse: 2.60682	valid_1's rmse: 2.93608
[1400]	training's rmse: 2.59212	valid_1's rmse: 2.9367
Early stopping, best iteration is:
[913]	training's rmse: 2.67931	valid_1's rmse: 2.9253
Our val mae score is 1.2295555157882077
WI_3
(4494582, 46)


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 2.70734	valid_1's rmse: 2.18218
[200]	training's rmse: 2.62798	valid_1's rmse: 2.1103
[300]	training's rmse: 2.56513	valid_1's rmse: 2.0826
[400]	training's rmse: 2.51285	valid_1's rmse: 2.06468
[500]	training's rmse: 2.46975	valid_1's rmse: 2.04922
[600]	training's rmse: 2.43697	valid_1's rmse: 2.038
[700]	training's rmse: 2.40517	valid_1's rmse: 2.03343
[800]	training's rmse: 2.38469	valid_1's rmse: 2.03167
[900]	training's rmse: 2.36464	valid_1's rmse: 2.02755
[1000]	training's rmse: 2.34251	valid_1's rmse: 2.02295
[1100]	training's rmse: 2.32419	valid_1's rmse: 2.0198
[1200]	training's rmse: 2.30863	valid_1's rmse: 2.01969
[1300]	training's rmse: 2.29548	valid_1's rmse: 2.02022
[1400]	training's rmse: 2.28142	valid_1's rmse: 2.01565
[1500]	training's rmse: 2.2702	valid_1's rmse: 2.0183
[1600]	training's rmse: 2.25788	valid_1's rmse: 2.01955
[1700]	training's rmse: 2.24643	valid_1's rmse: 2.0206
[1

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.


In [27]:
np.sqrt(np.mean((pred_true['pred'] - pred_true['true'])**2) )

2.1865670163093953

In [24]:
np.mean(np.abs(pred_true['pred'] - pred_true['true']) )

1.0556048913351244

In [0]:
validation = validation.set_index('id')
validation = validation.reindex(index=validation_csv['id'])
validation = validation.reset_index()

In [12]:
validation

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.707403,0.778371,0.593790,0.456425,0.626791,0.703639,0.602513,0.460977,0.668885,0.618535,0.639019,0.803799,1.161904,0.944817,0.798482,0.864328,0.591014,0.567763,0.663576,1.116052,0.814979,0.601054,0.725357,0.654348,0.678919,0.825208,1.131731,0.845007
1,HOBBIES_1_002_CA_1_validation,0.316021,0.323793,0.352226,0.380675,0.383175,0.411728,0.299425,0.200208,0.182671,0.195781,0.198145,0.209061,0.244914,0.198030,0.182671,0.182671,0.197098,0.202863,0.210986,0.246257,0.214940,0.183899,0.183899,0.263589,0.273050,0.289828,0.342081,0.284620
2,HOBBIES_1_003_CA_1_validation,0.315624,0.306150,0.320101,0.338020,0.378747,0.347340,0.306777,0.292556,0.289072,0.279680,0.333299,0.388020,0.640442,0.330636,0.282370,0.292425,0.315269,0.405163,0.448954,0.770122,0.523203,0.468318,0.467286,0.494236,0.507160,0.555128,0.832543,0.513696
3,HOBBIES_1_004_CA_1_validation,2.012819,1.589135,1.470378,1.539739,1.601337,2.064468,2.354625,1.243190,1.152885,1.147753,1.333391,1.477234,2.480738,2.848146,1.809254,1.497564,1.540723,1.672484,1.683169,2.152572,2.650333,1.614948,1.439450,1.263546,1.407295,1.637714,2.855996,3.286810
4,HOBBIES_1_005_CA_1_validation,0.887128,0.820169,0.930010,1.033608,1.112851,1.413475,1.622516,1.246557,1.194341,1.195186,1.016760,0.982402,1.450273,1.161164,0.883201,0.819446,0.908448,0.941478,1.053298,1.582011,1.472831,0.968174,0.851733,0.971658,1.004830,1.155752,1.647235,1.677827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,0.279038,0.229139,0.243871,0.215726,0.252780,0.300002,0.351593,0.293106,0.255480,0.243871,0.242320,0.269741,0.315626,0.351510,0.474787,0.338837,0.451999,0.348555,0.463227,0.522267,0.592744,0.470733,0.299586,0.353508,0.365595,0.396779,0.386759,0.486415
30486,FOODS_3_824_WI_3_validation,0.241716,0.223359,0.229032,0.226612,0.279657,0.350886,0.409190,0.488290,0.470240,0.424102,0.432911,0.418971,0.485819,0.535718,0.511448,0.426144,0.471741,0.456332,0.416480,0.423140,0.433380,0.383933,0.249070,0.229032,0.226612,0.279657,0.350886,0.385731
30487,FOODS_3_825_WI_3_validation,0.812388,0.668772,0.641803,0.540163,0.704668,0.830977,0.816040,0.861878,0.836141,0.841839,0.972350,1.011688,1.299377,1.311430,1.439858,0.959416,0.963124,0.818955,0.986815,1.371101,1.252650,0.872561,0.707059,0.714272,0.632937,0.811150,0.923961,0.837245
30488,FOODS_3_826_WI_3_validation,1.101860,1.077503,1.043995,0.836861,0.840419,1.068974,1.036571,0.759211,0.877082,0.676111,0.743221,0.822997,1.044233,1.385063,0.949027,0.921681,0.767653,0.873922,0.807876,1.017214,1.181225,0.710651,0.771827,0.644880,0.612079,0.677224,0.979613,1.286847


In [0]:
final = pd.concat([validation, evaluation])

In [0]:
submission_folder = '/content/drive/My Drive/Kaggle_M5/04_submissions/'
final.to_csv(submission_folder  + 'submission-by_store.csv', index = False)